# Mobile Customer Churn

In this Portfolio task you will work with some (fake but realistic) data on Mobile Customer Churn.  Churn is where
a customer leaves the mobile provider.   The goal is to build a simple predictive model to predict churn from available features. 

The data was generated (by Hume Winzar at Macquarie) based on a real dataset provided by Optus.  The data is simulated but the column headings are the same. (Note that I'm not sure if all of the real relationships in this data are preserved so you need to be cautious in interpreting the results of your analysis here).  

The data is provided in file `MobileCustomerChurn.csv` and column headings are defined in a file `MobileChurnDataDictionary.csv` (store these in the `files` folder in your project).

Your high level goal in this notebook is to try to build and evaluate a __predictive model for churn__ - predict the value of the CHURN_IND field in the data from some of the other fields.  Note that the three `RECON` fields should not be used as they indicate whether the customer reconnected after having churned. 

__Note:__ you are not being evaluated on the _accuracy_ of the model but on the _process_ that you use to generate it.  You can use a simple model such as Logistic Regression for this task or try one of the more advanced methods covered in recent weeks.  Explore the data, build a model using a selection of features and then do some work on finding out which features provide the most accurate results.  

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE
import warnings
warnings.filterwarnings("ignore")

In [10]:
churn = pd.read_csv("files/MobileCustomerChurn.csv", na_values=["NA", "#VALUE!"], index_col='INDEX')
churn.head()

,CUST_ID,ACCOUNT_TENURE,ACCT_CNT_SERVICES,AGE,CFU,SERVICE_TENURE,PLAN_ACCESS_FEE,BYO_PLAN_STATUS,PLAN_TENURE,MONTHS_OF_CONTRACT_REMAINING,...,CONTRACT_STATUS,PREV_CONTRACT_DURATION,HANDSET_USED_BRAND,CHURN_IND,MONTHLY_SPEND,COUNTRY_METRO_REGION,STATE,RECON_SMS_NEXT_MTH,RECON_TELE_NEXT_MTH,RECON_EMAIL_NEXT_MTH
INDEX,,,,,,,,,,,,,,,,,,,,,
1,1,46,1,30.0,CONSUMER,46,54.54,NON BYO,15,0,...,OFF-CONTRACT,24,SAMSUNG,1,61.40,COUNTRY,WA,NaN,NaN,NaN
2,2,60,3,55.0,CONSUMER,59,54.54,NON BYO,5,0,...,OFF-CONTRACT,24,APPLE,1,54.54,METRO,NSW,NaN,NaN,NaN
3,5,65,1,29.0,CONSUMER,65,40.90,BYO,15,0,...,OFF-CONTRACT,12,APPLE,1,2.50,COUNTRY,WA,NaN,NaN,NaN
4,6,31,1,51.0,CONSUMER,31,31.81,NON BYO,31,0,...,OFF-CONTRACT,24,APPLE,1,6.48,COUNTRY,VIC,NaN,NaN,NaN
5,8,95,1,31.0,CONSUMER,95,54.54,NON BYO,0,0,...,OFF-CONTRACT,24,APPLE,1,100.22,METRO,NSW,NaN,NaN,NaN


In [11]:
churn.shape

(46206, 21)

**DATA CLEANING**

This section will check for missing values in churn and I will eliminate these missing values if I feel they will influence the overall conclusion of the data.

In [12]:
# Print full summary
churn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46206 entries, 1 to 46206
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   CUST_ID                       46206 non-null  int64  
 1   ACCOUNT_TENURE                46206 non-null  int64  
 2   ACCT_CNT_SERVICES             46206 non-null  int64  
 3   AGE                           46130 non-null  float64
 4   CFU                           46206 non-null  object 
 5   SERVICE_TENURE                46206 non-null  int64  
 6   PLAN_ACCESS_FEE               46206 non-null  float64
 7   BYO_PLAN_STATUS               46206 non-null  object 
 8   PLAN_TENURE                   46206 non-null  int64  
 9   MONTHS_OF_CONTRACT_REMAINING  46206 non-null  int64  
 10  LAST_FX_CONTRACT_DURATION     46206 non-null  int64  
 11  CONTRACT_STATUS               46206 non-null  object 
 12  PREV_CONTRACT_DURATION        46206 non-null  int64  
 13  H

In [13]:
# Return the number of missing values in the data set
churn.isnull().sum()

CUST_ID                             0
ACCOUNT_TENURE                      0
ACCT_CNT_SERVICES                   0
AGE                                76
CFU                                 0
SERVICE_TENURE                      0
PLAN_ACCESS_FEE                     0
BYO_PLAN_STATUS                     0
PLAN_TENURE                         0
MONTHS_OF_CONTRACT_REMAINING        0
LAST_FX_CONTRACT_DURATION           0
CONTRACT_STATUS                     0
PREV_CONTRACT_DURATION              0
HANDSET_USED_BRAND                  0
CHURN_IND                           0
MONTHLY_SPEND                       0
COUNTRY_METRO_REGION                1
STATE                               1
RECON_SMS_NEXT_MTH              17790
RECON_TELE_NEXT_MTH             17790
RECON_EMAIL_NEXT_MTH            17790
dtype: int64

AGE, COUNTRY_METRO_REGION, STATE, RECON_SMS_NEXT_MTH, RECON_TELE_NEXT_MTH and RECON_EMAIL_NEXT_MTH are the only columns with missing values. With just one null value, COUNTRY_METRO_REGION and STATE will have little influence on the data; however, AGE has 76 missing values and RECON_SMS_NEXT_MTH, RECON_TELE_NEXT_MTH, and RECON_EMAIL_NEXT MTH all have 17790 missing values. To avoid errors due to null values, I will drop rows containing these nulls.

NaN values should be removed as part of the data cleaning process. This was taken care of in the following line:

In [33]:
# Remove rows and columns with Null/NaN values
churn.dropna(inplace = True)

Checking to see whether the columns satisfy expectations.

In [34]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28366 entries, 8153 to 46206
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   CUST_ID                       28366 non-null  int64  
 1   ACCOUNT_TENURE                28366 non-null  int64  
 2   ACCT_CNT_SERVICES             28366 non-null  int64  
 3   AGE                           28366 non-null  float64
 4   CFU                           28366 non-null  object 
 5   SERVICE_TENURE                28366 non-null  int64  
 6   PLAN_ACCESS_FEE               28366 non-null  float64
 7   BYO_PLAN_STATUS               28366 non-null  object 
 8   PLAN_TENURE                   28366 non-null  int64  
 9   MONTHS_OF_CONTRACT_REMAINING  28366 non-null  int64  
 10  LAST_FX_CONTRACT_DURATION     28366 non-null  int64  
 11  CONTRACT_STATUS               28366 non-null  object 
 12  PREV_CONTRACT_DURATION        28366 non-null  int64  
 13

Displaying cleaned data

In [35]:
churn.head()

,CUST_ID,ACCOUNT_TENURE,ACCT_CNT_SERVICES,AGE,CFU,SERVICE_TENURE,PLAN_ACCESS_FEE,BYO_PLAN_STATUS,PLAN_TENURE,MONTHS_OF_CONTRACT_REMAINING,...,CONTRACT_STATUS,PREV_CONTRACT_DURATION,HANDSET_USED_BRAND,CHURN_IND,MONTHLY_SPEND,COUNTRY_METRO_REGION,STATE,RECON_SMS_NEXT_MTH,RECON_TELE_NEXT_MTH,RECON_EMAIL_NEXT_MTH
INDEX,,,,,,,,,,,,,,,,,,,,,
8153,18301,15,1,66.0,CONSUMER,15,31.81,NON BYO,15,9,...,ON-CONTRACT,0,APPLE,0,74.17,COUNTRY,WA,0.0,0.0,0.0
8155,18304,49,2,55.0,CONSUMER,49,45.44,NON BYO,29,0,...,OFF-CONTRACT,24,SAMSUNG,0,89.60,METRO,NSW,0.0,0.0,0.0
8159,18312,71,2,34.0,CONSUMER,51,72.72,NON BYO,29,0,...,OFF-CONTRACT,24,SAMSUNG,0,161.12,COUNTRY,NSW,0.0,0.0,0.0
8169,18324,9,1,27.0,SMALL BUSINESS,9,72.72,NON BYO,9,15,...,ON-CONTRACT,24,SAMSUNG,0,63.62,METRO,VIC,0.0,0.0,0.0
8172,18328,46,1,34.0,CONSUMER,46,72.72,NON BYO,7,17,...,ON-CONTRACT,24,APPLE,0,84.53,METRO,VIC,0.0,0.0,0.0


**EXPLORATORY DATA ANALYSIS**

Now that the two datasets has been cleaned, we can now build and evaluate a predictive model for churn